More tools for Analyzing Grid Connectivity: Substation/Bus tools
=======

(For background information, take a look at IntroGridConnectivityTools notebook first.)

We are building a **graph-like representation of the electrical grid**. One interesting, and apparently underused, feature of grids is the ability to assign high-voltage lines to different buses on a substation. In the `grid2op` framework, each substation consists of up to 2 buses that can be used to assign generators or loads (externals), or lines. 

Nominally, all lines and externals are assigned to the same bus. However, different bus configurations can result in dramatically different grid behavior, and can in theory be used to cure grid issues such as line overloads.

Bus Configurations: Cutting Through the Combinatorics
----------
Nominally, it would seem like the number of bus configurations for a substation with $n$ connected elements is $2^{n}$; namely, each element can be connected to bus 1 or bus 2. You can make a simple symmetry argument to reduce this by a factor of 2: exchanging the bus labels $1\leftrightarrow 2$ you leads to the same configuration. With this in mind, the number of configurations drops to $2^{n-1}$.

Furthermore, we consider it invalid if a single line is connected to a bus (which has some intuitive sense). This reduces the number of valid configurations further, to $2^{n-1}-n$ ($n>2$). Note that, with these rules in place, buses with 2 or 3 elements only have 1 configuration option, e.g. they are essentially static.

But there are additional ways that bus configurations become invalid (or double-counted) when lines are disconnected. For instance, if a line is disconnected, then it is ambiguous whether it would have been assigned to bus 1 or bus 2. (**It is important to note here that line disconnection accounting is performed at the line level, not at the bus level.**) Therefore a bus configuration 1001*0* and 1001*1* (where the bold element is disconnected) are degenerate.

In this next segment, we have several goals:
 - We need to define a **way to label a bus configuration** in a way that is unique.
 - We want to develop a check to determine if a certain bus configuration is **valid or invalid**.
 - Once we have these, we can determine **how many bus configurations are possible**, given some number of connected elements.
- We also need an accounting mechanism to keep track of **degenerate bus configurations** in case of disconnected elements.

To define a unique label, we represent a bus configuration with a bitset, e.g. 0b1010 for elements 0 and 2 connected to "bus 0", and lines 1 and 3 connected to "bus 1". (Note that the leading "0b" is meant to indicate in the code that this is a binary number.) A bitwise representation is useful because we can use efficient bitwise operations to determine bus configuration validity. (Note that we are abandoning the "bus 1" and "bus 2" notation, in favor of 0 and 1.)

The bitwise representation of bus configurations allows us to impose restrictions that dictate the validity of a given bus configuration, in order to enforce symmetry and degeneracy rules:
 - By convention, we require the leftmost bit to be connected to bus 1. This avoids double-counting cases with $0\leftrightarrow1$ symmetry. Therefore e.g. 0b1100 is labeled as valid, while the degenerate 0b0011 is called invalid for counting purposes.
 - We consider illegal the case where a single line is connected to a bus -- e.g. 0b1110 is invalid.
 - When a line is disconnected, the number of possible bus configurations for an "N-line bus" should reduce to the number of possible configurations for an "N-minus-one-line bus". To avoid double-counting identical bitsets in these cases, valid bitsets must have the disconnected line(s) assigned to "bus 1". In reality, of course, they are not connected at all, but we explicitly want to label as invalid the bitsets with the disconnected line(s) assigned to "bus 0" to keep the valid bus states unique.
 
We implement these rules with functions in a module, whose results are shown below:

In [ ]:
# This helper module is mine:
import BusTopologyHelpers

The function `IsValidBooleanBusState(flag,nbits)` checks whether a boolean bus state (flag) is valid (you must also pass the number of total externals (bits). Let's check some of our examples above:

In [ ]:
BOLD = '\033[1m'
END = '\033[0m'
print(BOLD+'0 <--> 1 Symmetry:'+END)
print('Leftmost bit is 1: 0b{:0{}b} is {}'.format(0b10001,5,BusTopologyHelpers.IsValidBooleanBusState(0b10001,5)))
print('Leftmost bit is 0: 0b{:0{}b} is {}'.format(0b00011,5,BusTopologyHelpers.IsValidBooleanBusState(0b00011,5)))
print()
print(BOLD+'Single-line-on-a-bus rule:'+END)
print('One line on a bus: 0b{:0{}b} is {}'.format(0b11110,5,BusTopologyHelpers.IsValidBooleanBusState(0b11110,5)))

We can check all possible configurations, given a number of externals:

In [ ]:
# Try up to 9 externals
for j in range(1,10) :
    ValidBusStates = list(BusTopologyHelpers.IsValidBooleanBusState(i,j) for i in range(pow(2,j)))
    print('Valid configurations in {}-externals case:'.format(j),sum(ValidBusStates))

Finally, we want to be able to disconnect $m$ lines and check the number of remaining possible configurations, which should be equal to the number of configurations in the $n-m$ case. We do this using the `items_disconnected` argument of the `IsValidBooleanBusState` function (see below).

We try this with 8 externals (lines), three of which are disconnected, and try to confirm that only 11 valid configurations remain.

In [ ]:
items_disconnected = [0,1,2]
nEdges = 8
print('\nInvestigating case with {} lines, with these lines disconnected:'.format(nEdges),items_disconnected)
n = 0
for i in range(pow(2,nEdges)) :
    isValid = BusTopologyHelpers.IsValidBooleanBusState(i,nEdges,items_disconnected=items_disconnected)
    if not isValid :
        continue
    print('Flag 0b{:0{}b} is {}'.format(i,nEdges,'legal' if isValid else 'illegal'))
    n += 1
print('For {} edges, there are {} configurations that are legal,'.format(nEdges,n),
      'given these disconnected items:',items_disconnected)